This notebook is an inference script of the models created from a TPU training script.

Training : 
- The training script is using only TPU and TFRecord . The official tfrecord seems to have some issue, as my models was stuck during training to a local minimal. https://www.kaggle.com/ludovick/baseline-tf-tpu-efficientnet-kfold-training
- The data augmentation and the TPU code comes from these two notebooks : https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords and https://www.kaggle.com/jessemostipak/getting-started-tpus-cassava-leaf-disease
- the current possible augmentation are : rotation, shift, zoom, shear, crop, hue etc.
- tfrecord stratified: https://www.kaggle.com/ludovick/cassavatfrecords512x512q100
- mixed precision is used : bfloat16 for training

Inference :
- use for now a generator to do the prediction on the test data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#! pip install ../input/efficientnet-keras-source-code
import sys
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')
! pip install -e /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle

In [ ]:


import math, re, os
import random
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)
import efficientnet.tfkeras as efn
from sklearn.metrics import accuracy_score
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    DEVICE = "TPU"
except:
    DEVICE = "notTPU"
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:

cfg = {"smoothing":0.00,
       "arch_fn":efn,
      "name": [f"EfficientNetB{i}" for i in [4,5,6]],
      "num_class":5,
       "epochs":20,
       "kfold":[5,5,5],
       "seed":42,
       "verbose":1,
      # augmentation
       "resize":512,
       "crop_size":480,
       "rotation":0.0,
       "shear":0.0,
       "h-zoom":5.0,
       "w-zoom":5.0,
       "h-shift":5.0,
       "w-shift":5.0,
       
      "path_models":"../input/cassavatpumodelsbaseline/"
      }
AUTOTUNE = tf.data.experimental.AUTOTUNE
# GCS_PATH = KaggleDatasets().get_gcs_path("cassava-leaf-disease-classification")

REPLICAS =  strategy.num_replicas_in_sync
FILENAMES = tf.io.gfile.glob("../input/cassava-leaf-disease-classification" + '/test_tfrecords/ld_test*.tfrec')
BATCH_SIZE = 128 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]

In [ ]:
print(FILENAMES)

In [ ]:
os.environ['PYTHONHASHSEED']=str(cfg["seed"])
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
random.seed(cfg["seed"])
np.random.seed(cfg["seed"])
tf.random.set_seed(cfg["seed"])

# load data

In [ ]:
def read_tfrecord(example, labeled, return_image_name=False):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        if return_image_name:
            return image, tf.reshape(tf.one_hot([label], depth=cfg["num_class"], axis=-1), [-1]), example["image_name"]
        return image, tf.reshape(tf.one_hot([label], depth=cfg["num_class"], axis=-1), [-1])
    idnum = example['image_name']
    return image, idnum

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False, return_image_name=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled, return_image_name=return_image_name), num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(filenames, ordered=True, tta=False):
    dataset = load_dataset(filenames, labeled=False, ordered=ordered)
    if tta:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# Augmentation

In [ ]:
# https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, cfg):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = cfg["resize"]
    ROT_ = cfg["rotation"]
    SHR_ = cfg["shear"]
    HZOOM_ = cfg["h-zoom"]
    WZOOM_ = cfg["w-zoom"]
    HSHIFT_ = cfg["h-shift"]
    WSHIFT_ = cfg["w-shift"]
    
    
    
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def data_augment(img, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
    global cfg

    img = transform(img, cfg) # 
    img = tf.image.random_crop(img, [cfg['crop_size'], cfg['crop_size'], 3])
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_hue(img, 0.01)
    img = tf.image.random_saturation(img, 0.7, 1.3)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.image.random_brightness(img, 0.1)
    img = tf.image.resize(img, [cfg["resize"], cfg["resize"]] )
    return img, label

# Model

In [ ]:
def get_model(cfg, name):
    model_input = tf.keras.Input(shape=(cfg['resize'], cfg['resize'], 3), name='inputs')
    constructor = getattr(cfg["arch_fn"], name)
    x = constructor(include_top=False, weights=None, 
                        input_shape=(cfg['resize'], cfg['resize'], 3), 
                        pooling=None)(model_input)
    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    outputs = tf.keras.layers.Dense(cfg["num_class"], activation='softmax', name="outputs")(x)
    model = tf.keras.Model(model_input, outputs, name=name+'_{0}'.format(cfg["resize"]))
    #model.summary()
    return model

def compile_new_model(cfg, name):    
    with strategy.scope():
        model = get_model(cfg, name)

        losses = tf.keras.losses.CategoricalCrossentropy(label_smoothing = cfg['smoothing'])
        model.compile(
            optimizer = tf.keras.optimizers.Adam(lr=1e-3),
            loss      = losses,
            metrics   = tf.keras.metrics.CategoricalAccuracy()
        )
        
    return model

# Inference

In [ ]:
JPEG_PATH = "../input/cassava-leaf-disease-classification/test_images"
JPEG_PATH_TR = "../input/cassava-leaf-disease-classification/train_images"

import cv2
from tqdm.notebook import tqdm
def load_image(jpeg_path, image_id):
    img = cv2.imread(os.path.join(jpeg_path, image_id))/255.0
    img = cv2.resize(img, (cfg["resize"], cfg["resize"]))[:, :, ::-1]

    return img
def generator(filepath, paths, batch_size=32):
    i=0
    print(len(paths))
    while i <= len(paths):
        batch = []
        for cpt in range(batch_size):
            if i + cpt >= len(paths):
                i += batch_size
                break
            batch.append(load_image(filepath, paths[i+cpt]))
            
        i += batch_size
        yield np.stack(batch)

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
tr = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
preds_all = []
for num_model in range(len(cfg["name"])):
    preds_model = []
    for fold in range(cfg["kfold"][num_model]):
        print(f"## FOLD: {fold}")
        
        ds_test = generator(JPEG_PATH,submission.image_id.values)#get_test_dataset(FILENAMES, tta=False)
        #ds_test = generator(JPEG_PATH_TR, tr.image_id.values)

        # BUILD MODEL
        K.clear_session()
        with strategy.scope():
            model = compile_new_model(cfg, cfg['name'][num_model])

        print('Loading best model...')
        model.load_weights(cfg["path_models"]+ cfg['name'][num_model] + "/" + cfg['name'][num_model]+'-fold-%i.h5'%fold)

        # prediction on val

        preds = model.predict(ds_test, verbose=True)
        preds_model.append(preds)                 

    preds_model = np.stack(preds_model).mean(0)
    preds_all.append(preds_model)
    
preds_all = np.stack(preds_all)

In [ ]:
preds_all.shape

In [ ]:
"""
ds_test = get_test_dataset(FILENAMES, tta=False)

test_ids_ds = ds_test.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(BATCH_SIZE))).numpy().astype('U')
"""

In [ ]:
preds_all

In [ ]:
#submission["image_id"] = test_ids
submission["label"] = preds_all.mean(0).argmax(1)
submission.to_csv("submission.csv", index=False)

In [ ]:
submission